In [145]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import json
from datetime import datetime 
from alpaca_trade_api.rest import REST, TimeFrame
import tweepy
%matplotlib inline
from pathlib import Path
import csv
import re
import emoji
import nltk

In [131]:
# Load .env enviroment variables
load_dotenv()
bearer_token =  os.getenv('TWITTER_BEARER_TOKEN')
api_key =  os.getenv('TWITTER_API_KEY')
api_secret_key =  os.getenv('TWITTER_SECRET_KEY')
access_token =  os.getenv('TWITTER_ACCESS_TOKEN')
access_secret_token =  os.getenv('TWITTER_SECRET_TOKEN')


def get_api_keys():
    api_key =  os.getenv('TWITTER_API_KEY')
    api_secret_key =  os.getenv('TWITTER_SECRET_KEY')
    bearer_token =  os.getenv('TWITTER_BEARER_TOKEN')
    access_token =  os.getenv('TWITTER_ACCESS_TOKEN')
    access_secret_token =  os.getenv('TWITTER_SECRET_TOKEN')
    return api_key,api_secret_key,bearer_token,access_token,access_secret_token

def get_api_keys1():
    api_key =  os.getenv('access_token')
    api_secret_key =  os.getenv('access_token_secret')
    bearer_token =  os.getenv('TWITTER_BEARER_TOKEN')
    access_token =  os.getenv('consumer_key')
    access_secret_token =  os.getenv('consumer_secret')
    return api_key,api_secret_key,bearer_token,access_token,access_secret_token

api_key,api_secret_key,bearer_token,access_token,access_secret_token = get_api_keys()


In [130]:
url = "https://api.twitter.com/2/tweets/1535608320211427329?tweet.fields=created_at,attachments&expansions=author_id"

payload={}
headers = {
  'id': '1228393702244134912',
  'Authorization': 'Bearer '+ bearer_token,
  'Cookie': 'guest_id=v1%3A165495549908649152'
}

response = requests.request("GET", url, headers=headers, data=payload)
json_response = response.text
print(json_response)

{"data":{"text":"On this day in 1980: Steve Ballmer became Microsoft employee #24.\n\nHis shares are now worth $81 billion. https://t.co/tHc8wtldIG","author_id":"164425558","attachments":{"media_keys":["3_1535608317837443073"]},"created_at":"2022-06-11T13:02:08.000Z","id":"1535608320211427329"},"includes":{"users":[{"id":"164425558","name":"Jon Erlichman","username":"JonErlichman"}]}}


In [134]:
query = "BAYC"
client = tweepy.Client(bearer_token=bearer_token)
counts = client.get_recent_tweets_count(query=query,granularity="day")
for count in counts.data:
    print(count)

{'end': '2022-06-07T00:00:00.000Z', 'start': '2022-06-06T23:36:49.000Z', 'tweet_count': 251}
{'end': '2022-06-08T00:00:00.000Z', 'start': '2022-06-07T00:00:00.000Z', 'tweet_count': 48509}
{'end': '2022-06-09T00:00:00.000Z', 'start': '2022-06-08T00:00:00.000Z', 'tweet_count': 21305}
{'end': '2022-06-10T00:00:00.000Z', 'start': '2022-06-09T00:00:00.000Z', 'tweet_count': 20500}
{'end': '2022-06-11T00:00:00.000Z', 'start': '2022-06-10T00:00:00.000Z', 'tweet_count': 46016}
{'end': '2022-06-12T00:00:00.000Z', 'start': '2022-06-11T00:00:00.000Z', 'tweet_count': 24803}
{'end': '2022-06-13T00:00:00.000Z', 'start': '2022-06-12T00:00:00.000Z', 'tweet_count': 40333}
{'end': '2022-06-13T23:36:49.000Z', 'start': '2022-06-13T00:00:00.000Z', 'tweet_count': 28305}


In [136]:
def create_tweet(text):
    client = tweepy.Client(consumer_key=api_key,
        consumer_secret=api_secret_key,
        access_token=access_token,
        access_token_secret=access_secret_token)
    response=client.create_tweet(text=text)

#create_tweet("Hello World!")

In [39]:
def determine_file_name(query):
    file_name = "Data/"+query+".csv"
    return file_name

In [40]:
def search_tweets(query):
    client = tweepy.Client(bearer_token=bearer_token)
    response = client.search_recent_tweets(query=query)
    return response

query = "NFT"

response = search_tweets(query)
#for tweet in response.data:
   # print(tweet.text)

In [114]:
#this function writes the event data to a csv file to avoid calling the API multiple times
def write_csv(data_list, file_name):
    with open(file_name, mode='w', encoding='utf-8', newline='\n') as f:
        header = "Tweets"
        f.write("%s\n" % header)
        for data in data_list:
            f.write("%s\n" % data)

In [173]:
def remove_emoji(string):
    return emoji.get_emoji_regexp().sub(u'', string)

nltk.download('words')
words = set(nltk.corpus.words.words())


def remove_non_english_words(text):
    return " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in words or not w.isalpha())

def sanitize_tweet(tweet):
    sanitzed_tweet = tweet.replace("\n", " ")
    sanitzed_tweet = re.sub(r'@[A-za-z0-9]+','',sanitzed_tweet)
    sanitzed_tweet = re.sub(r'#','',sanitzed_tweet)
    sanitzed_tweet = re.sub(r'RT[\s]','',sanitzed_tweet)
    sanitzed_tweet = re.sub(r'https?:\/\/\S+','',sanitzed_tweet)
    sanitzed_tweet = re.sub(r',+','',sanitzed_tweet)
    #sanitzed_tweet = re.sub(r'✅+','',sanitzed_tweet)
    #sanitzed_tweet = re.sub(r'🚨+','',sanitzed_tweet)
    #sanitzed_tweet = remove_emoji(sanitzed_tweet)
    #sanitzed_tweet = remove_non_english_words(sanitzed_tweet)
    sanitzed_tweet =  re.sub('[^A-Za-z0-9]+', ' ', sanitzed_tweet)
    return sanitzed_tweet

def remove_duplicates(tweet_list):
    unique_lines = set()
    for tweet in tweet_list:
        unique_lines.add(tweet)
    return list(unique_lines)  

[nltk_data] Downloading package words to
[nltk_data]     /Users/sameerlakhe/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [176]:
#get the max data
limit = 50

max_results = 100
tweet_list=[]
def search_tweets1(query,reload=False):
    client = tweepy.Client(bearer_token=bearer_token)
    file_name = determine_file_name(query)
    path = Path(file_name)
    
    i=1
    if (reload == False and path.is_file()):
        print(f'The file {file_name} exists')
        nft_df = pd.read_csv(file_name)
        return nft_df
    else:
        for tweet in tweepy.Paginator(client.search_recent_tweets,query=query,max_results=max_results).flatten(limit=limit):
            #print(str(i) +")" + tweet.text)
            i=i+1
            sanitized_tweet = sanitize_tweet(tweet.text)
            tweet_list.append(sanitized_tweet)
        #once the list is generated, write it to file
        unique_tweet_list = remove_duplicates(tweet_list)
        write_csv(unique_tweet_list,file_name)
    #return list as a dataframe
    return pd.DataFrame(unique_tweet_list)

query = "BAYC"
is_reload_needed = True
tweet_df = search_tweets1(query,is_reload_needed)

pd.set_option('max_colwidth', 800)
tweet_df


,0
0,Bayc mayc or whatever they call monkeys w wigs amp clothes
1,bayc better than goblins
2,bayc for the win
3,Only 24 hours until we announce the 500 AirDrop Winners Sharing with your favorite BAYC Holders will improve your chan
4,UPDATED TODAY MSC will break thin walls Big announcement coming soon The majority of MSC holders are actual ent
5,MORE MERRCCHH BAYC MAYC NFT
6,BAYC d
7,Otherdeed 61927 bought for 2 12 2519 09 otherside otherdeed BAYC MAYC APE Metaverse WhaleAnalytica
8,BAYC
9,BIGGEST Giveaway 3 Free BAYC NFT 50k 10000 in ETH Rules 1 Follow amp Me 2 Like 3 Tag 2 fr


In [133]:
query = "#NFT"
is_reload_needed = False
tweet_df = search_tweets1(query,is_reload_needed)
tweet_df

,0
0,: SHIBURAI and VIRALCRYPTO are prime examples ...
1,: 0.4 ETH ($1000) NFT Giveaway 🥽🥽 To enter ...
2,Free NFT
3,: Will we see the bulls regain control for a g...
4,: 🔥 BIG WL GIVEAWAY 🔥 2x WL for each Collecti...
...,...
1495,: Win 5X OG Roles! 🌙 To Enter👇🏻 1. + Like 2....
1496,セルシオ、ビットコインの出金を停止。何が問題だったのか？ iOSアプリでもっと...
1497,: 🪶 𝐍𝐅𝐓 𝐆𝐈𝐕𝐄𝐀𝐖𝐀𝐘 🪶 🏆 10WL spots 🏆 1 $SOL ($...
1498,: Good Morning Nugz 🌳 Let's kill it today! ...
